In [53]:
import pandas as pd
import numpy as np
import researchpy as rp
from scipy import stats

import matplotlib.pyplot as plt
import pydotplus
from matplotlib import rcParams
import seaborn as sb

In [54]:
# # Find no. of factors in y
from collections import Counter
# #label encoding
from sklearn.preprocessing import LabelEncoder, scale, MinMaxScaler, StandardScaler,OneHotEncoder,LabelBinarizer
#from sklearn.decomposition import PCA

In [55]:
# Import All classifcation Algorithms
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [56]:
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score,validation_curve,KFold

In [57]:
# Metrics selection
from sklearn.metrics import  accuracy_score, recall_score, confusion_matrix,precision_recall_fscore_support,classification_report,roc_curve

In [58]:
# read data file
df1 = pd.read_csv('2_SalesEffect_Cleaned_Encoded.csv')
print(df1.shape)

(6574, 6)


In [59]:
X1 = df1.iloc[:,:-1]
y1 = df1.y_Status

In [60]:
# Split 70:30
split_seed = 10
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,random_state=split_seed, test_size=.3)

# Scaling done based on EDA analysis. Fit on the Train set, but tranform both transform both training and test set 

In [61]:
# Create Column Lists
# Create All lists
AllColumns = ['c_Product_ID', 'c_Source','c_Sales_Agent', 
              'c_Location', 'c_Delivery_Mode', 'y_Status']

# No Continuous Columns
ConVarList=[]

CatVarList = ['c_Product_ID', 'c_Source','c_Sales_Agent', 
              'c_Location', 'c_Delivery_Mode']

OutcomeVar=['y_Status']

# Initialize
#scaler = StandardScaler()
# Fit on train data
#scaler.fit(X1_train[ConVarList])
# Transform both train and test
#X1_train[ConVarList] = scaler.transform(X1_train[ConVarList])
#X1_test[ConVarList] = scaler.transform(X1_test[ConVarList])

# Optimization of hyperparameters using Grid search  and random state variations

In [47]:
parameters = {"learning_rate": [.01,.02,.03,.04,.05],
              "max_depth": [3,5],
              "min_child_weight" : [ 2,3 ],
              "gamma"            : [ 0.1],
              "colsample_bytree" : [ 0.7], 
              "scoring"         : ["accuracy"] 
             }

model = XGBClassifier(booster='gbtree',random_state=10)
grid_model_XGB = GridSearchCV(model, parameters)
grid_model_XGB.fit(X1_train, y1_train)

E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05], 'max_depth': [3, 5], 'min_child_weight': [2, 3], 'gamma': [0.1], 'colsample_bytree': [0.7], 'scoring': ['accuracy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [48]:
grid_model_XGB.best_params_

{'colsample_bytree': 0.7,
 'gamma': 0.1,
 'learning_rate': 0.05,
 'max_depth': 5,
 'min_child_weight': 2,
 'scoring': 'accuracy'}

In [49]:
y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test),columns=['y_Status'])
print(confusion_matrix(y1_test,y1_predict))
print(accuracy_score(y1_test,y1_predict).round(4)*100)
print(classification_report(y1_test,y1_predict))

[[998  76  11  18]
 [249 139   2  17]
 [138  31  13  12]
 [157  54   3  55]]
61.07
              precision    recall  f1-score   support

           0       0.65      0.90      0.75      1103
           1       0.46      0.34      0.39       407
           2       0.45      0.07      0.12       194
           3       0.54      0.20      0.30       269

   micro avg       0.61      0.61      0.61      1973
   macro avg       0.52      0.38      0.39      1973
weighted avg       0.57      0.61      0.55      1973



# Vary Model Random state to see if the results can be further improved

In [31]:
parameters = {"learning_rate": [.05],
              "max_depth": [5],
              "min_child_weight" : [2],
              "gamma"            : [0.1],
              "colsample_bytree" : [.7], 
              "scoring"         : ["accuracy"] 
             }
store = []
store_max=0
for i in range(10,500,5):
    model = XGBClassifier(booster='gbtree',random_state=i)
    grid_model_XGB = GridSearchCV(model, parameters)
    grid_model_XGB.fit(X1_train, y1_train)
    y1_predict = grid_model_XGB.predict(X1_test)
    if(accuracy_score(y1_test,y1_predict)>=store_max):
        store.append([i,accuracy_score(y1_test,y1_predict)*100])
        store_max=accuracy_score(y1_test,y1_predict)
    print(i,accuracy_score(y1_test,y1_predict)*100)


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


10 61.07450582868728


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


15 61.581348200709584


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


...

485 61.73340091231627


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


490 61.784085149518496


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


495 61.73340091231627


In [33]:
store

[[10, 61.07450582868728],
 [15, 61.581348200709584],
 [20, 61.73340091231627],
 [25, 62.24024328433857],
 [155, 62.290927521540794]]

# Gradient Boosting Classifier

In [50]:
grid_model_XGB = GradientBoostingClassifier(random_state=10)
grid_model_XGB.fit(X1_train, y1_train)

y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test),columns=['y_Status'])
print(confusion_matrix(y1_test,y1_predict))
print(accuracy_score(y1_test,y1_predict).round(4)*100)
print(classification_report(y1_test,y1_predict))

[[988  79  17  19]
 [232 146   8  21]
 [129  33  21  11]
 [139  45  15  70]]
62.09
              precision    recall  f1-score   support

           0       0.66      0.90      0.76      1103
           1       0.48      0.36      0.41       407
           2       0.34      0.11      0.16       194
           3       0.58      0.26      0.36       269

   micro avg       0.62      0.62      0.62      1973
   macro avg       0.52      0.41      0.42      1973
weighted avg       0.58      0.62      0.58      1973



# Final Model

In [62]:
model = XGBClassifier(booster='gbtree',
                      random_state=155,
                      learning_rate=0.05,
                      max_depth=5,
                      min_child_weight=2,
                      gamma = 0.1,
                      colsample_bytree=0.7,
                      scoring='accuracy'
                     )
model.fit(X1_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0.1, learning_rate=0.05,
       max_delta_step=0, max_depth=5, min_child_weight=2, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=155, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, scoring='accuracy', seed=None,
       silent=True, subsample=1)

In [63]:
# Results
y1_predict = pd.DataFrame(model.predict(X1_test),columns=['y_Status'])
print(confusion_matrix(y1_test,y1_predict))
print(accuracy_score(y1_test,y1_predict).round(4)*100)
print(classification_report(y1_test,y1_predict))

[[1003   78    7   15]
 [ 235  155    1   16]
 [ 139   32   13   10]
 [ 152   56    3   58]]
62.29
              precision    recall  f1-score   support

           0       0.66      0.91      0.76      1103
           1       0.48      0.38      0.43       407
           2       0.54      0.07      0.12       194
           3       0.59      0.22      0.32       269

   micro avg       0.62      0.62      0.62      1973
   macro avg       0.57      0.39      0.41      1973
weighted avg       0.60      0.62      0.57      1973



# Check Underfit / Overfit symptom

In [64]:
y1_predict = pd.DataFrame(model.predict(X1_train),columns=['y_Status'])
print(confusion_matrix(y1_train,y1_predict))
print(accuracy_score(y1_train,y1_predict).round(4)*100)
print(classification_report(y1_train,y1_predict))

#Since Train Accuracy 66.16 > test accuracy 62.29, there is a case of Some Raltive overfit, 
#we can look at increasing test train split to further improve the test accuracy.
# Please note the symptom of low accuracy had a symptom in feature engg file, where we fould very abysmal correlation of 
# predictors on the outcome variable.

[[2457  136   17   14]
 [ 515  399    2   27]
 [ 323   68   40   20]
 [ 310  116    9  148]]
66.16
              precision    recall  f1-score   support

           0       0.68      0.94      0.79      2624
           1       0.55      0.42      0.48       943
           2       0.59      0.09      0.15       451
           3       0.71      0.25      0.37       583

   micro avg       0.66      0.66      0.66      4601
   macro avg       0.63      0.43      0.45      4601
weighted avg       0.65      0.66      0.61      4601

